In [1]:
import glob
import pathlib

import pandas as pd
import plotly.express as px

from btree_analysis.log_parser import parse_experiment

In [20]:
EXPERIMENT_PATH = "/Users/artur/Documents/broadcast-tree/experiments/results/*"
EXPERIMENT_IDENTIFIER = ["payload_size", "max_power", "unchanged_counter", "iteration"]

In [2]:
events = []
for experiment in glob.glob(EXPERIMENT_PATH):
    _events, _, _ = parse_experiment(pathlib.Path(experiment))
    events += _events
    
df = pd.DataFrame(events)

Node 77 switched parents 1 times
Node 1 switched parents 6 times
Node 51 switched parents 1 times
Node 8 switched parents 1 times
Node 60 switched parents 0 times
Node 44 switched parents 1 times
Node 37 switched parents 1 times
Node 23 switched parents 1 times
Node 42 switched parents 0 times
Node 50 switched parents 0 times
Node 75 switched parents 1 times
Node 58 switched parents 1 times
Node 32 switched parents 1 times
Node 26 switched parents 1 times
Node 38 switched parents 1 times
Node 45 switched parents 2 times
Node 18 switched parents 2 times
Node 79 switched parents 1 times
Node 0 switched parents 0 times
Node 68 switched parents 1 times
Node 28 switched parents 1 times
Node 72 switched parents 4 times
Node 40 switched parents 2 times
Node 41 switched parents 1 times
Node 54 switched parents 1 times
Node 83 switched parents 1 times
Node 25 switched parents 0 times
Node 49 switched parents 1 times
Node 57 switched parents 1 times
Node 76 switched parents 1 times
Node 24 switc

# Evaluation Metrics
- [x] Table: How many nodes have received the entire payload?
- [x] Plot: Percentage comparison of BTP vs. max_power w.r.t. to received payloads
- [X] Plot: How many percent more or less energy does BTP need compared to max_power
- [X] Plot: What is the overhead of the BTP protocol (i.e. payload energy vs. protocol energy)?
- [] Plot: How are individual nodes strained?

# Absolute Values of Received Payloads

In [21]:
reception_df = df.groupby(EXPERIMENT_IDENTIFIER)["event"].apply(lambda x: (x == 'receive').sum()).reset_index(name='count')

In [22]:
reception_df

,payload_size,max_power,unchanged_counter,iteration,count
0,100K,False,5,0,2
1,100K,False,15,0,2
2,100K,False,25,0,7
3,100K,True,5,0,14
4,100K,True,15,0,12
5,100K,True,25,0,12
6,1K,False,5,0,65
7,1K,False,15,0,73
8,1K,False,25,0,71
9,1K,True,5,0,65


In [5]:
fig = px.box(
    reception_df,
    x="payload_size",
    y="count", 
    color="unchanged_counter",
    facet_col="max_power",
)
fig

# Relative Values of Received Payloads

In [44]:
def reception_ratio(x):
    flooding = x[x["max_power"] == True]
    btp = x[x["max_power"] == False]
    
    return float(btp["count"]) / flooding["count"]

count_df = reception_df.groupby(["payload_size", "unchanged_counter", "iteration"]).apply(reception_ratio).reset_index(name='ratio')

In [45]:
fig = px.box(
    count_df,
    x="payload_size",
    y="ratio", 
    color="unchanged_counter",
)
fig

# Prepare energy df

In [90]:
def energy_ratio(x):
    energy_payload = x[x["frame_type"] == "payload"]
    energy_protocol = x[x["frame_type"] == "protocol"]
    
    total_energy = float(energy_payload["mWb_total"]) + float(energy_protocol["mWb_total"])
    
    payload_rel = float(energy_payload["mWb_total"]) / total_energy
    protocol_rel = float(energy_protocol["mWb_total"]) / total_energy
    
    
    x.loc[x["frame_type"] == "payload", "mWb_rel"] = payload_rel
    x.loc[x["frame_type"] == "protocol", "mWb_rel"] = protocol_rel
    
    return x
    
    

df["mW"] = 10**(df["tx_pwr"]/10)
df["mWb"] = df["mW"] * (df["data_len"] + 34)

df.loc[(df["event"] == "send") & (df["message"] == "Successfully forwarded payload.") | (df["message"] == "Successfully sent next chunk."), "frame_type"] = "payload"
df.loc[(df["event"] == "send") & ((df["message"] != "Successfully forwarded payload.") & (df["message"] != "Successfully sent next chunk.")), "frame_type"] = "protocol"

energy_df = df[df["event"] == "send"].groupby(EXPERIMENT_IDENTIFIER + ["frame_type"])["mWb"].sum().reset_index(name='mWb_total')
energy_df = energy_df.groupby(EXPERIMENT_IDENTIFIER).apply(energy_ratio).reset_index()


# Comparison between flooding and BTP, energy wise

In [117]:
def energy_ratio(x):    
    flooding = x[(x["max_power"] == True) & (x["frame_type"] == "payload")]
    btp = x[(x["max_power"] == False)].sum()
    
    return btp["mWb_total"] / flooding["mWb_total"]

energy_ratio_df = energy_df.groupby(["payload_size", "unchanged_counter", "iteration"]).apply(energy_ratio).reset_index(name='ratio')

In [118]:
fig = px.box(
    energy_ratio_df,
    x="payload_size",
    y="ratio", 
    color="unchanged_counter",
)
fig

# What is BTP's overhead for building the tree

In [124]:
btp_energy_df = energy_df[energy_df["max_power"] == False]

fig = px.bar(
    btp_energy_df,
    x="payload_size",
    y="mWb_rel",
    color="frame_type",
    facet_col="unchanged_counter",
)
fig

# Strain on Individual Nodes

In [187]:
def sort_groups(x):
    sorted_x = x.sort_values(by="amount", ascending=False)
    sorted_x['sorted'] = range(len(sorted_x))
    
    return sorted_x

strain_df = df.drop(df[(df["max_power"] == True) & (df["frame_type"] == "protocol")].index)

strain_df = strain_df.groupby(EXPERIMENT_IDENTIFIER + ["node_id"])["mWb"].sum().reset_index(name="amount")

strain_df = strain_df.groupby(EXPERIMENT_IDENTIFIER).agg(sort_groups).reset_index(drop=True)

strain_df["normalized"] = (strain_df["amount"] - strain_df["amount"].min()) / (strain_df["amount"].max() - strain_df["amount"].min())

/var/folders/26/2pp7vp553ms3s26tctkklxzr0000gp/T/ipykernel_39347/37051079.py:11: FutureWarning:

Dropping invalid columns in DataFrameGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the function.



In [188]:
fig = px.line(
    strain_df,
    x="sorted",
    y="normalized",
    color="max_power",
    facet_col="unchanged_counter",
    line_dash="payload_size"
)
fig